In [9]:
import json
import os
import random
from coco_help import *

In [10]:
def get_im_info(im_id, json_path):
    # open json
    with open(json_path, 'r') as f:
        gt = json.load(f)
    
    # find your image
    for i in gt['images']:
        if i['id'] == im_id:
            return i
    return None

In [11]:
def coco_to_yolo_bbox(im_id, im_info, coco_bbox):
    
    # Get image info and extract image width and height
    im_h = im_info['height']
    im_w = im_info['width']
    
    # Get components of original coco bbox
    [x1, y1, w, h] = coco_bbox
    
    # Get center point using x1, y1, w, h
    xc = x1 + (w/2)
    yc = y1 + (h/2)
    
    # Scale by image height and width to be a percentage
    xc_y = xc/im_w
    yc_y = yc/im_h
    w_y = w/im_w
    h_y = h/im_h
    
    yolo_bbox = f'{xc_y} {yc_y} {w_y} {h_y}'
    
    return yolo_bbox

In [12]:
def coco_to_yolo_anns(im_id, im_anns, json_path):
    with open(json_path, 'r') as f:
        gt = json.load(f)
        
    im_info = get_im_info(im_id, json_path)
    
    yolo_anns = []
    for c_ann in im_anns:
        coco_bbox = c_ann['bbox']
        y_box = coco_to_yolo_bbox(im_id, im_info, coco_bbox)
        cat_id = c_ann['category_id']
        y_ann = f'{cat_id} {y_box}'
        yolo_anns.append(y_ann)
    return yolo_anns

In [13]:
def make_yolo_anns_from_coco(ann_dir, json_path):
    
    with open(json_path, 'r') as f:
        gt = json.load(f)
        
    if not os.path.exists(ann_dir):
        os.mkdir(ann_dir)

    image_paths = os.listdir(train_images)
    
    print(f'{len(image_paths)} images in folder')
    count = 0
    
    for i in image_paths:
        im_id = int(i.split('_')[0])
        im_anns = anns_on_image(im_id, json_path)
        yolo_anns = coco_to_yolo_anns(im_id, im_anns, json_path)
        img_txt = ann_dir + i.replace('.png', '.txt')
        if os.path.exists(img_txt):
            os.remove(img_txt)
        with open(img_txt, 'w') as f:
            for line in yolo_anns:
                f.write(line + '\n')
        count  += 1
        if count%100 ==0:
            print(f'{count}/{len(image_paths)} files processed')
    return

In [14]:
cwd = os.getcwd()
cwd = cwd.replace('\\', '/') + '/'

train_images = cwd + 'val_1000/images_1000/'
train_json = cwd + 'val_1000/COCO_1000.json'

new_yolo_anns = cwd + 'val_1000/yolo_train_1000/'

make_yolo_anns_from_coco(new_yolo_anns, train_json)

2680 images in folder
100/2680 files processed
200/2680 files processed
300/2680 files processed
400/2680 files processed
500/2680 files processed
600/2680 files processed
700/2680 files processed
800/2680 files processed
900/2680 files processed
1000/2680 files processed
1100/2680 files processed
1200/2680 files processed
1300/2680 files processed
1400/2680 files processed
1500/2680 files processed
1600/2680 files processed
1700/2680 files processed
1800/2680 files processed
1900/2680 files processed
2000/2680 files processed
2100/2680 files processed
2200/2680 files processed
2300/2680 files processed
2400/2680 files processed
2500/2680 files processed
2600/2680 files processed


In [17]:
def make_cats(json_path):
    # open current coco json
    with open(json_path, 'r') as f:
        gt = json.load(f)
    # get categories - the important section
    cats = gt['categories']
    
    cat_list = []
    for c in cats:
        cat_list.append([c['id'], c['name']])
    cat_list.sort()
    
    # Open new file
    new_path = 'classes.names'
    # make sure it doesn't already exist
    if os.path.exists(new_path):
        os.remove(new_path)
    # open sesame
    with open(new_path, 'w') as cats:
        for c in cat_list:
            cats.write(c[1] + '\n')
        
    return

In [18]:
make_cats(train_json)